In [2]:
!ls

YBIGTA_Transfer_Learning.ipynb
data
image_crawler.ipynb
keras 튜토리얼 (국문).ipynb
labels_test.csv
labels_train.csv


# Preparation

(아래 코드들 설명)
1. `original_data.zip` 파일을 다운로드 한다. (카톡방 참고)
2. 아래 코드를 실행시키고 `original_data.zip`을 업로드 한다.
3. 압축을 푼다. (미견도사, 진돗개, 삽살개 각각 하나의 폴더가 존재함)
4. `train_test_ratio` 를 정하고 `random.random()` 을 돌려 각 데이터를 `/train` 폴더나 `/test` 폴더에 복사한다
5. 각 폴더(`train`/`test`)에 들어있는 데이터를 바탕으로 csv를 만든다.



In [3]:
# 2. 아래 코드를 실행시키고 original_data.zip을 업로드 한다.
from google.colab import files
files.upload()  # original_data.zip

ModuleNotFoundError: No module named 'google.colab'

In [0]:
# 3. 압축을 푼다. (미견도사, 진돗개, 삽살개 각각 하나의 폴더가 존재함)
!unzip original_copies.zip

In [4]:
# If data/train and data/test folders exist already, skip this block.
!mkdir data
%cd data
!mkdir train test
%cd ..

mkdir: data: File exists
/Users/yuyejin/Documents/동아리/Ybigta/프로젝트/data
/Users/yuyejin/Documents/동아리/Ybigta/프로젝트


In [51]:
import tensorflow as tf

# image rename
classes = ['dosamastiff', 'jindotgae', 'sapsalgae']
for c in classes:
    dirpath = os.path.join('data', c) #data/c 로 경로 설정
    files = os.listdir(dirpath)
    i = 1
    for f in files:
        fpath = os.path.join(dirpath, f)
        newpath = os.path.join('dataset', f'{c}_{i}.jpg')
        os.rename(fpath, newpath)
        i += 1

In [14]:
# 4. train_test_ratio 를 정하고 random.random() 을 돌려 각 데이터를 /train 폴더나 /test 폴더에 복사한다

import os
from os.path import join
import random
import shutil

# For reproducibility
# random.seed(42)

classes = ['dosamastiff', 'jindotgae', 'sapsalgae']
train_test_split = 0.2

for c in classes:
  dirpath = join('data', c) #data/c 로 경로 설정
  files = os.listdir(dirpath)
  for f in files:
    fpath = join(dirpath, f)
    folder = 'test' if random.random() < train_test_split else 'train'
    new_dir = join('data', folder)
    shutil.copy(fpath, new_dir)

In [18]:
# 5. 각 폴더(train/test)에 들어있는 데이터를 바탕으로 csv를 만든다.
import pandas as pd

for folder in ['train', 'test']:
  fnames = os.listdir(join('data', folder))
  ids = pd.Series(fnames)
  labels = list(ids.map(lambda x: x.split('_')[0]))
  df = pd.DataFrame({'id':ids, 'breed':labels})
  df.to_csv(f'labels_{folder}.csv',index=False)

# Preprocessing

In [19]:
import keras as k
import numpy as np
import pandas as pd
import os

from keras.preprocessing import image

from keras.applications import xception

NUM_CLS = 3
SIZE = (299,299)

def img_and_label(img_id, train_test, size):
    cls = img_id.split('_')[0]
    img = image.load_img(os.path.join('data', train_test, img_id), target_size=size)
    img = image.img_to_array(img)
    return img, cls

In [20]:
labels_train = pd.read_csv('labels_train.csv')
X_train = np.zeros((len(labels_train), *SIZE, 3))
y_train = np.zeros((len(labels_train), NUM_CLS))

labels_test = pd.read_csv('labels_test.csv')
X_test = np.zeros((len(labels_test), *SIZE, 3))
y_test = np.zeros(len(labels_test))

y_template = {'dosamastiff':0, 'jindotgae':1, 'sapsalgae':2}

In [22]:
labels_train.head(10)

,breed,id
0,dosamastiff,dosamastiff_1.jpg
1,dosamastiff,dosamastiff_100.jpg
2,dosamastiff,dosamastiff_101.jpg
3,dosamastiff,dosamastiff_102.jpg
4,dosamastiff,dosamastiff_103.jpg
5,dosamastiff,dosamastiff_104.jpg
6,dosamastiff,dosamastiff_106.jpg
7,dosamastiff,dosamastiff_107.jpg
8,dosamastiff,dosamastiff_108.jpg
9,dosamastiff,dosamastiff_109.jpg


In [23]:
len(os.listdir('data/jindotgae')) + len(os.listdir('data/sapsalgae')) + len(os.listdir('data/dosamastiff'))

493

In [24]:
for i, fname in enumerate(labels_train['id']):
    img, label = img_and_label(fname, 'train', SIZE)
    x = xception.preprocess_input(img[np.newaxis,:,:,:])
    X_train[i] = x
    
    # y_train needs to be one-hot
    y_train[i, y_template[label]] = 1
    
for i, fname in enumerate(labels_test['id']):
    img, label = img_and_label(fname, 'test', SIZE)
    x = xception.preprocess_input(img[np.newaxis,:,:,:])
    X_test[i] = x
    
    # y_test need not be one-hot
    y_test[i] = y_template[label]

In [121]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(389, 299, 299, 3)
(389, 3)
(104, 299, 299, 3)
(104,)


In [27]:
y_train[:7]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

# Transfer Learning

In [29]:
# Data agmentation
datagen = image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
datagen.fit(X_train)

In [0]:
POOLING = 'avg'
BATCH_SIZE = 32
NUM_EPOCHS = 30


base_model = xception.Xception(include_top=False, weights='imagenet', input_shape=(299,299,3), pooling=POOLING)
x = base_model.output
pred = k.layers.Dense(3, activation='softmax')(x)

model = k.models.Model(inputs=base_model.input, outputs=pred)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy')

print('Model loaded and compiled. Now starting training...')

model.fit_generator(
    datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
    steps_per_epoch= int(len(X_train) / BATCH_SIZE),
    epochs=NUM_EPOCHS)

In [0]:
pred = model.predict(X_test)
pred_vec = np.argmax(pred, axis=1)

In [0]:
# Accuracy
np.mean(y_test == pred_vec)

In [0]:
# 작성중

import matplotlib as mpl
import matplotlib.pyplot as plt

wrong_idx = (y_test != prev_vec)
wrong_data = labels_test[wrong_idx]

# Visualise 16 wrong images
wrong_ids = wrong_data.head(16)['id']
wrong_labels = wrong_data.head(16)['labels']

# CNN Neural Net

In [67]:
type(X_train[0][0][0][0])

numpy.float64

In [68]:
type(X_train[0][0][0][0])

numpy.ndarray

In [63]:
temp = X_train

In [69]:
temp.astype('float32')

array([[[[ 0.9764706 ,  0.9764706 ,  0.9764706 ],
         [ 0.9764706 ,  0.9764706 ,  0.9764706 ],
         [ 0.9764706 ,  0.9764706 ,  0.9764706 ],
         ...,
         [ 0.9764706 ,  0.9764706 ,  0.9764706 ],
         [ 0.9764706 ,  0.9764706 ,  0.9764706 ],
         [ 0.9764706 ,  0.9764706 ,  0.9764706 ]],

        [[ 0.9764706 ,  0.9764706 ,  0.9764706 ],
         [ 0.9764706 ,  0.9764706 ,  0.9764706 ],
         [ 0.9764706 ,  0.9764706 ,  0.9764706 ],
         ...,
         [ 0.9764706 ,  0.9764706 ,  0.9764706 ],
         [ 0.9764706 ,  0.9764706 ,  0.9764706 ],
         [ 0.9764706 ,  0.9764706 ,  0.9764706 ]],

        [[ 0.9764706 ,  0.9764706 ,  0.9764706 ],
         [ 0.9764706 ,  0.9764706 ,  0.9764706 ],
         [ 0.9764706 ,  0.9764706 ,  0.9764706 ],
         ...,
         [ 0.9764706 ,  0.9764706 ,  0.9764706 ],
         [ 0.9764706 ,  0.9764706 ,  0.9764706 ],
         [ 0.9764706 ,  0.9764706 ,  0.9764706 ]],

        ...,

        [[ 0.9764706 ,  0.9764706 ,  0

In [116]:
temp[0][0][0][0] = temp[0][0][0][0].astype('float32')
print(type(temp[0][0][0][0]))
print(type(temp[0][0][0][0].astype('float32')))

<class 'numpy.float64'>
<class 'numpy.float32'>


In [109]:
for a in range(0, len(temp)):
    for b in range(0, len(temp[a])):
        for c in range(0, len(temp[a][b])):
            for d in range(0, len(temp[a][b][c])):
                temp[a][b][c][d] = temp[a][b][c][d].astype('float32')
                print(type(temp[a][b][c][d]))

In [38]:
y_train

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [30]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [52]:
# CNN 모델을 정의합니다. 
def cifar(x):
    
      """CIFAR-10 이미지를 분류하기 위한 Convolutional Neural Networks 그래프를 생성합니다.
      인자들(Args):
        x: (N_examples, 32, 32, 3) 차원을 가진 input tensor, CIFAR-10 데이터는 32x32 크기의 컬러이미지이다.
      리턴값들(Returns):
        tuple (y, keep_prob). y는 (N_examples, 10)형태의 숫자(0-9) tensor이다. 
        keep_prob는 dropout을 위한 scalar placeholder이다.
      """

      # 입력 이미지
      x_image = x

      # 첫번째 convolutional layer - 하나의 grayscale 이미지를 64개의 특징들(feature)으로 맵핑(maping)한다.
      W_conv1 = tf.Variable(tf.truncated_normal(shape=[5, 5, 3, 64], stddev=5e-2))
      b_conv1 = tf.Variable(tf.constant(0.1, shape=[64]))
      h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1)

      # 첫번째 Pooling layer
      h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

      # 두번째 convolutional layer -- 32개의 특징들(feature)을 64개의 특징들(feature)로 맵핑(maping)한다.
      W_conv2 = tf.Variable(tf.truncated_normal(shape=[5, 5, 64, 64], stddev=5e-2))
      b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
      h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2)

      # 두번째 pooling layer.
      h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

      # 세번째 convolutional layer
      W_conv3 = tf.Variable(tf.truncated_normal(shape=[3, 3, 64, 128], stddev=5e-2))
      b_conv3 = tf.Variable(tf.constant(0.1, shape=[128]))
      h_conv3 = tf.nn.relu(tf.nn.conv2d(h_pool2, W_conv3, strides=[1, 1, 1, 1], padding='SAME') + b_conv3)

      # 네번째 convolutional layer
      W_conv4 = tf.Variable(tf.truncated_normal(shape=[3, 3, 128, 128], stddev=5e-2))
      b_conv4 = tf.Variable(tf.constant(0.1, shape=[128])) 
      h_conv4 = tf.nn.relu(tf.nn.conv2d(h_conv3, W_conv4, strides=[1, 1, 1, 1], padding='SAME') + b_conv4)

      # 다섯번째 convolutional layer
      W_conv5 = tf.Variable(tf.truncated_normal(shape=[3, 3, 128, 128], stddev=5e-2))
      b_conv5 = tf.Variable(tf.constant(0.1, shape=[128]))
      h_conv5 = tf.nn.relu(tf.nn.conv2d(h_conv4, W_conv5, strides=[1, 1, 1, 1], padding='SAME') + b_conv5)

      # Fully Connected Layer 1 -- 2번의 downsampling 이후에, 우리의 32x32 이미지는 8x8x128 특징맵(feature map)이 된다.
      # 이를 384개의 특징들로 맵핑(maping)한다.
      W_fc1 = tf.Variable(tf.truncated_normal(shape=[8 * 8 * 128, 384], stddev=5e-2))
      b_fc1 = tf.Variable(tf.constant(0.1, shape=[384]))

      h_conv5_flat = tf.reshape(h_conv5, [-1, 8*8*128])
      h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

      # Dropout - 모델의 복잡도를 컨트롤한다. 특징들의 co-adaptation을 방지한다.
      h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob) 

      # 384개의 특징들(feature)을 10개의 클래스-airplane, automobile, bird...-로 맵핑(maping)한다.
      W_fc2 = tf.Variable(tf.truncated_normal(shape=[384, 10], stddev=5e-2))
      b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))
      logits = tf.matmul(h_fc1_drop,W_fc2) + b_fc2
      y_pred = tf.nn.softmax(logits)

      return y_pred, logits

In [123]:
# CNN 모델을 정의합니다. 
def build_CNN_classifier():
    
      # input
      x_image = tf.placeholder(tf.float32, [None, 299, 299, 3])

      # 첫번째 convolutional layer 
      W_conv1 = tf.Variable(tf.truncated_normal(shape=[5, 5, 3, 64], stddev=5e-2))
      b_conv1 = tf.Variable(tf.constant(0.1, shape=[64]))
        
      h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1)

      # 첫번째 Pooling layer
      h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

      # 두번째 convolutional layer 
      W_conv2 = tf.Variable(tf.truncated_normal(shape=[5, 5, 64, 64], stddev=5e-2))
      b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
      h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2)

      # 두번째 pooling layer.
      h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

      # 세번째 convolutional layer
      W_conv3 = tf.Variable(tf.truncated_normal(shape=[3, 3, 64, 128], stddev=5e-2))
      b_conv3 = tf.Variable(tf.constant(0.1, shape=[128]))
      h_conv3 = tf.nn.relu(tf.nn.conv2d(h_pool2, W_conv3, strides=[1, 1, 1, 1], padding='SAME') + b_conv3)

      # 네번째 convolutional layer
      W_conv4 = tf.Variable(tf.truncated_normal(shape=[3, 3, 128, 128], stddev=5e-2))
      b_conv4 = tf.Variable(tf.constant(0.1, shape=[128])) 
      h_conv4 = tf.nn.relu(tf.nn.conv2d(h_conv3, W_conv4, strides=[1, 1, 1, 1], padding='SAME') + b_conv4)

      # 다섯번째 convolutional layer
      W_conv5 = tf.Variable(tf.truncated_normal(shape=[3, 3, 128, 128], stddev=5e-2))
      b_conv5 = tf.Variable(tf.constant(0.1, shape=[128]))
      h_conv5 = tf.nn.relu(tf.nn.conv2d(h_conv4, W_conv5, strides=[1, 1, 1, 1], padding='SAME') + b_conv5)

      # Fully Connected Layer 1
      W_fc1 = tf.Variable(tf.truncated_normal(shape=[8 * 8 * 128, 384], stddev=5e-2))
      b_fc1 = tf.Variable(tf.constant(0.1, shape=[384]))

      h_conv5_flat = tf.reshape(h_conv5, [-1, 8*8*128])
      h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat, W_fc1) + b_fc1)

      # Dropout 
      h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob) 

      W_fc2 = tf.Variable(tf.truncated_normal(shape=[384, 3], stddev=5e-2))
      b_fc2 = tf.Variable(tf.constant(0.1, shape=[3]))
      logits = tf.matmul(h_fc1_drop,W_fc2) + b_fc2
      y_pred = tf.nn.softmax(logits)

      return x_image, y_pred, logits

In [124]:
# scalar 형태의 레이블(0~9)을 One-hot Encoding 형태로 변환합니다.
#y_train_one_hot = tf.squeeze(tf.one_hot(y_train, 10),axis=1)
#y_test_one_hot = tf.squeeze(tf.one_hot(y_test, 10),axis=1)

# Convolutional Neural Networks(CNN) 그래프를 생성합니다.
x_images, y_pred, logits = build_CNN_classifier()

# Cross Entropy를 비용함수(loss function)으로 정의하고, RMSPropOptimizer를 이용해서 비용 함수를 최소화합니다.
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
train_step = tf.train.RMSPropOptimizer(1e-3).minimize(loss)

# 정확도를 계산하는 연산을 추가합니다.
correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [ ]:
# 세션을 열어 실제 학습을 진행합니다.
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(y_pred, feed_dict={x_image: X_train})
  
    # 10000 Step만큼 최적화를 수행합니다.
    for i in range(10000):
        batch = next_batch(128, X_train, y_train())

    # 100 Step마다 training 데이터셋에 대한 정확도와 loss를 출력합니다.
    if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={X: batch[0], y: batch[1], keep_prob: 1.0})
        loss_print = loss.eval(feed_dict={X: batch[0], y: batch[1], keep_prob: 1.0})

        print("반복(Epoch): %d, 트레이닝 데이터 정확도: %f, 손실 함수(loss): %f" % (i, train_accuracy, loss_print))
        print("Current time is : ", datetime.datetime.now())
    # 20% 확률의 Dropout을 이용해서 학습을 진행한다.
    sess.run(train_step, feed_dict={X: batch[0], y: batch[1], keep_prob: 0.8})

    # 학습이 끝나면 테스트 데이터에 대한 정확도를 출력합니다.  
    test_batch = next_batch(10000, X_test, y_test)
    print("테스트 데이터 정확도: %f" % accuracy.eval(feed_dict={X: test_batch[0], y: test_batch[1], keep_prob: 1.0}))